In [1]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from datetime import datetime

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [4]:
driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)

# 멜론 차트


## 월별




In [ ]:
# 2021.3월 차트 (현재 기준 최신 월별 차트)
# Get HTML using Webdriver with BeautifulSoup
url = 'https://www.melon.com/chart/month/index.htm'
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
# Get songs top 100 within a month
songs = soup.select('div.ellipsis.rank01')
singers = soup.select('div.ellipsis.rank02 > span.checkEllipsis')

# Make a DataFrame
list_song = [song.text.strip('\n') for song in songs]
list_singer = [singer.text for singer in singers]
data = pd.DataFrame(data=zip(range(1,101),list_song, list_singer), columns=['Rank', 'Title', 'Singer'])
print(data.head())

# Close webdriver
driver.close()
driver.quit()

# Save a DataFrame to a Excel File(.xlsx)
now = datetime.now()
filename = now.strftime('Melon_month_Top100_at_%Y%m%d_%Hh%Mm.xlsx')
data.to_excel(filename, index=False)

## 시대별 (연도별)

In [5]:
import selenium
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat

In [6]:
driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)
driver.maximize_window()
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)



# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 연도차트 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[3]/a')
btn.click()
time.sleep(2)

# 2010년대 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[2]/span/label')
btn.click()
time.sleep(2)

# 2019년
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label')
btn.click()
time.sleep(2)

# 2018년   
# //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label

# 2017년
# //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[3]/span/label

# 2011년
# //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[9]/span/label

# 국내종합 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[2]/span/label')
btn.click()
time.sleep(2)

# 검색버튼 클릭

url = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()


In [7]:
html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
                            # cf) requests.get(url)
soup = BeautifulSoup(html, 'lxml')

In [8]:
title_list = [title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
title_list[:10]

['사랑에 연습이 있었다면 (Prod. 2soo)',
 '그때가 좋았어',
 '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)',
 '넘쳐흘러',
 '모든 날, 모든 순간 (Every day, Every Moment)',
 '너를 만나',
 '사계 (Four Seasons)',
 '주저하는 연인들을 위해',
 '니 소식',
 '오늘도 빛나는 너에게 (To You My Light) (Feat.이라온)']

In [9]:
singer_list = [singer.get_text() for singer in soup.find_all('span', attrs={'class':'checkEllipsis'})]
singer_list[:10]

['임재현',
 '케이시 (Kassy)',
 '방탄소년단',
 '엠씨더맥스 (M.C the MAX)',
 '폴킴',
 '폴킴',
 '태연 (TAEYEON)',
 '잔나비',
 '송하예',
 '마크툽 (MAKTUB)']

In [10]:
rank = [i for i in range(1, 101)]

In [12]:
df = pd.DataFrame({'순위':rank,'곡명':title_list,'가수명':singer_list})
# result_df = pd.concat([result_df, df], ignore_index=True)


In [13]:
df.to_csv('melon_chart_2019.csv', encoding='cp949')

In [ ]:
def melon_age_search(year): 
  driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)
  driver.maximize_window()
  url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
  driver.get(url)



  # 차트파인더 클릭
  driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

  # 연도차트 클릭
  btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[3]/a')
  btn.click()
  time.sleep(2)

  # 2010년대 클릭
  btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[2]/span/label')
  btn.click()
  time.sleep(2)

  # 2019년
  btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{}]/span/label'.format(2020-year))
  btn.click()
  time.sleep(2)

  # 2018년   
  # //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label

  # 2017년
  # //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[3]/span/label
  # ...
  # 2011년
  # //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[9]/span/label

  # 2010년
  # //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[10]/span/label

  # 국내종합 클릭
  btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[2]/span/label')
  btn.click()
  time.sleep(2)

  # 검색버튼 클릭

  url = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

  html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
                            # cf) requests.get(url)
  soup = BeautifulSoup(html, 'lxml')  
  title_list = [title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
  singer_list = [singer.get_text() for singer in soup.find_all('span', attrs={'class':'checkEllipsis'})]
  rank = [i for i in range(1, 101)]

  df = pd.DataFrame({'순위':rank,'곡명':title_list,'가수명':singer_list})
  result_df = pd.concat([result_df, df], ignore_index=True)

  result_df.to_csv('melon_chart_{}.csv'.format(year), encoding='cp949')

  driver.close()
  driver.quit()


In [ ]:
"""for year in range(2017, 2020):
  melon_age_search(year)

"""

In [ ]:
# 2019
driver = webdriver.Chrome('/usr/bin/chromedriver', options=options)
driver.maximize_window()
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)



# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 연도차트 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[3]/a')
btn.click()
time.sleep(2)

# 2010년대 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[2]/span/label')
btn.click()
time.sleep(2)

# 2019년
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label')
btn.click()
time.sleep(2)


# 국내종합 클릭
btn = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[2]/span/label')
btn.click()
time.sleep(2)

# 검색버튼 클릭
driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

html = driver.page_source 
soup = BeautifulSoup(html, 'lxml')  

title_list = [title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
singer_list = [singer.get_text() for singer in soup.find_all('span', attrs={'class':'checkEllipsis'})]
rank = [i for i in range(1, 101)]

df = pd.DataFrame({'순위':rank,'곡명':title_list,'가수명':singer_list})
result_df = pd.concat([result_df, df], ignore_index=True)

result_df.to_csv('melon_chart_2019.csv', encoding='cp949')

driver.close()
driver.quit()


ValueError: ignored

# 빌보드 차트

## 현재 hot-100 차트

In [ ]:
# 현재 hot-100 차트

from bs4 import BeautifulSoup
import requests
import csv

csv_filename = "hot_chart100.csv"
csv_open = open(csv_filename, "w+", encoding='utf-8')
csv_writer = csv.writer(csv_open)
csv_writer.writerow( ('Rank', 'Song Title', 'Singer', ) )

req = requests.get('https://www.billboard.com/charts/hot-100')
html = req.text
soup = BeautifulSoup(html, 'html.parser')

rank_list = soup.select('.chart-element__rank__number')
song_list = soup.select('.chart-element__information__song')
singer_list = soup.select('.chart-element__information__artist')

for i in range(0, 100):
    csv_writer.writerow( (rank_list[i].text, song_list[i].text, singer_list[i].text, ) )
    # print(rank_list[i].text, song_list[i].text, singer_list[i].text)

csv_open.close()

## 연도별 hot 100

In [ ]:
# 연도별 hot 100
# 2019년 

from bs4 import BeautifulSoup
import requests
import csv

csv_filename = "hot_chart100_2019.csv"
csv_open = open(csv_filename, "w+", encoding='utf-8')
csv_writer = csv.writer(csv_open)
csv_writer.writerow( ('Rank', 'Song Title', 'Singer', ) )

req = requests.get('https://www.billboard.com/charts/year-end/2019/hot-100-songs')
html = req.text
soup = BeautifulSoup(html, 'html.parser')

rank_list = soup.select('.ye-chart-item__rank')
song_list = soup.select('.ye-chart-item__title')
singer_list = soup.select('.ye-chart-item__artist')

for i in range(0, 100):
    csv_writer.writerow((rank_list[i].text, song_list[i].text, singer_list[i].text,))
    # print(rank_list[i].text, song_list[i].text, singer_list[i].text)

csv_open.close()

[]

In [ ]:
rank_list = soup.select('.chart-element__rank__number')

In [ ]:
singer_list

[]